In [1]:
import pandas as pd

pd.set_option("display.max_columns", 150)
df2017 = pd.read_csv('./data/test2017.csv')
df2018 = pd.read_csv('./data/test2018.csv')
df2019 = pd.read_csv('./data/test2019.csv')
df2020 = pd.read_csv('./data/test2020.csv')
df2021 = pd.read_csv('./data/test2021.csv')

In [2]:
df_ls = [df2017, df2018, df2019, df2020, df2021]

In [3]:
df = df2017.rename(columns={'기준일ID':'ds','총생활인구수':'y'})
df = pd.concat([df, df2018.rename(columns={'기준일ID':'ds','총생활인구수':'y'})])
df = pd.concat([df, df2019.rename(columns={'기준일ID':'ds','총생활인구수':'y'})])
df = pd.concat([df, df2020.rename(columns={'기준일ID':'ds','총생활인구수':'y'})])
df = pd.concat([df, df2021.rename(columns={'기준일ID':'ds','총생활인구수':'y'})])

In [4]:
import datetime as dt

df['ds'] = pd.to_datetime((df['ds'].astype(str)+[' ']+df['시간대구분'].astype(str)+[':00:00']), format="%Y%m%d %H:%M:%S")
df.drop(['시간대구분','Unnamed: 0'],axis=1,inplace=True)

In [5]:
Q1 = df['y'].quantile(q=0.25)
Q3 = df['y'].quantile(q=0.75)
IQR = Q3-Q1

((df['y'] <= Q3+1.5*IQR) & (df['y'] >= Q1-1.5*IQR)).value_counts()

True     34666
False       86
Name: y, dtype: int64

In [206]:
# for i in df[((df['y'] <= Q3+1.5*IQR) & (df['y'] >= Q1-1.5*IQR))==False].index:
#     df.iloc[i,df.columns.get_loc('y')]=(df.iloc[i-10:i+10]['y'].mean())/10

In [207]:
# ((df['y'] <= Q3+1.5*IQR) & (df['y'] >= Q1-1.5*IQR)).value_counts()

In [208]:
# for i in df[df['y'] <= Q3+1.5*IQR].index:
#     df.iloc[i,df.columns.get_loc('y')]=Q3+1.5*IQR

# for i in df[df['y'] >= Q1+1.5*IQR].index:
#     df.iloc[i,df.columns.get_loc('y')]=Q1+1.5*IQR

In [5]:
import holidays
kr_holidays = holidays.KR()
df['holiday'] = df['ds'].apply(lambda x: 'holiday' if x in kr_holidays else 'non-holiday')

In [6]:
param_grid = {  'growth': ["linear"], 
                'changepoints': [None], 
                'n_changepoints': [25, 50, 75], 
                'changepoint_range': [0.25, 0.5, 0.75],
                'yearly_seasonality': ["auto"],
                'weekly_seasonality': ["auto"],
                'daily_seasonality': [False],
                'holidays': [holidays],
                'seasonality_mode': ["additive"],
                'seasonality_prior_scale': [10, 50, 100],
                'holidays_prior_scale': [10, 50, 100],
                'changepoint_prior_scale': [0.1, 0.33, 0.66],
                'mcmc_samples': [0],
                'interval_width': [0.25, 0.5, 0.75],
                'uncertainty_samples': [0]
              }

In [16]:
from prophet import Prophet

m = Prophet(weekly_seasonality=True, daily_seasonality = True, yearly_seasonality = True, interval_width=0.8).fit(df)

19:47:04 - cmdstanpy - INFO - Chain [1] start processing
19:49:01 - cmdstanpy - INFO - Chain [1] done processing


In [17]:
future = m.make_future_dataframe(periods=1416, freq='H')
fcst = m.predict(future)
# fig = m.plot(fcst)

In [18]:
fcst.iloc[-1416:]['yhat']

43512    38762.262396
43513    39161.069968
43514    39032.288785
43515    38912.231682
43516    39130.583204
             ...     
44923    34755.839679
44924    35137.192525
44925    35995.650113
44926    37413.476136
44927    38998.890096
Name: yhat, Length: 1416, dtype: float64

In [37]:
submission = pd.read_csv('./data/sample_submission.csv')

In [35]:
count = fcst.iloc[-1416:]['yhat']
count = pd.DataFrame(count)
count = count.reset_index(drop=True)
count

,yhat
0,38762.262396
1,39161.069968
2,39032.288785
3,38912.231682
4,39130.583204
...,...
1411,34755.839679
1412,35137.192525
1413,35995.650113
1414,37413.476136


In [38]:
submission['count'] = count['yhat']

In [40]:
submission 

,id,count
0,0,38762.262396
1,1,39161.069968
2,2,39032.288785
3,3,38912.231682
4,4,39130.583204
...,...,...
1411,1411,34755.839679
1412,1412,35137.192525
1413,1413,35995.650113
1414,1414,37413.476136


In [41]:
submission.to_csv("submission_1.csv", index = False)

In [10]:
df2021.drop(['Unnamed: 0','시간대구분'],axis=1,inplace=True)

In [11]:
import numpy as np
from sklearn.metrics import mean_squared_error

a = df2021.iloc[:1416]['총생활인구수'].to_list()
b = fcst.iloc[-1416:]['yhat'].to_list()

mean_squared_error(b, a)**0.5

1763.1911283388877